In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import surprise
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import Dataset, evaluate

from surprise import SVD
from surprise import SVDpp
from surprise import SlopeOne
from surprise import NMF
from surprise import NormalPredictor
from surprise import KNNBaseline
from surprise import KNNBasic 
from surprise import KNNWithZScore 
from surprise import BaselineOnly
from surprise import CoClustering
from surprise.model_selection import PredefinedKFold

from surprise.model_selection import train_test_split

data = pd.read_csv('ratings data.csv')

In [27]:
data = pd.read_csv('ratings data.csv')

In [3]:
#data.head()

In [28]:
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(data[['userID', 'itemID', 'rating']], reader)

In [29]:
trainingSet = data.build_full_trainset()

In [30]:
sim_options = {
    'name': 'cosine',
    'user_based': True
}
 
knn = KNNWithZScore (sim_options=sim_options)

In [31]:
#knn.train(trainingSet)
#testset = trainset.build_anti_testset()

In [32]:

trainset, testset = train_test_split(data, test_size=.25)
#predictions = knn.test(testset)
predictions = knn.fit(trainset).test(testset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [33]:
accuracy.rmse(predictions, verbose=True)

RMSE: 0.9851


0.9851451199247727

In [36]:
from surprise.model_selection import KFold
kf = KFold(n_splits=5)

#algo = KNNWithZScore()
bsl_options = {'method': 'sgd',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

Estimating biases using sgd...
RMSE: 0.9848
Estimating biases using sgd...
RMSE: 0.9853
Estimating biases using sgd...
RMSE: 0.9853
Estimating biases using sgd...
RMSE: 0.9856
Estimating biases using sgd...
RMSE: 0.9851


In [ ]:
#trainset = data.build_full_trainset()
#algo = SVD()
#algo.train(trainset)
#testset = data.construct_testset(raw_testset=data.raw_ratings)
#predictions = algo.test(testset)

In [ ]:
from collections import defaultdict
def get_top_k(predictions, k):
    '''Return a top_k dicts where keys are user ids and values are lists of
    tuples [(item id, rating estimation) ...].

    Takes in a list of predictions as returned by the test method.
    '''

    # First map the predictions to each user.
    top_k = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_k[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_k.items():
        user_ratings.sort(key=lambda x:x[1], reverse=True)
        top_k[uid] = user_ratings[:k]

    return top_k

In [ ]:
top_k = get_top_k(predictions, 10)

In [ ]:
# Print the recommended items
for uid, user_ratings in top_k.items():
    print(uid, [iid for (iid, _) in user_ratings])
# Compute the total number of recommended items.
all_recommended_items = set(iid for (_, user_ratings) in top_k.items() for
                            (iid, _) in user_ratings)
#print('Number of recommended items:', len(all_recommended_items), 'over',
      #len(top_k), 'users')

In [ ]:
accuracy.rmse(predictions, verbose=True)